# MUSIC GENERATION USING DEEP LEARNING

In [81]:
import pandas as pd
import numpy as np
import glob
from music21 import converter, instrument, note, chord, stream
import pickle
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Get Notes from Music File 

In [4]:
# converter convert into midi file 
midi = converter.parse('midi_songs/dontbeafraid.mid') 

In [5]:
midi #midi object bangaya stream.Score

<music21.stream.Score 0x7f4ab34285f8>

In [8]:
midi.show('midi')
midi.show('text')

{0.0} <music21.stream.Part 0x7f4ab3424a20>
    {0.0} <music21.instrument.Piano 'Piano'>
    {0.0} <music21.tempo.MetronomeMark Quarter=163.0>
    {0.0} <music21.tempo.MetronomeMark Quarter=163.0>
    {0.0} <music21.key.Key of d minor>
    {0.0} <music21.meter.TimeSignature 5/4>
    {0.0} <music21.stream.Voice 0x7f4ab3168518>
        {0.0} <music21.chord.Chord A3 D4>
        {1.0} <music21.chord.Chord A3 D4>
        {1.5} <music21.chord.Chord A3 D4>
        {2.5} <music21.chord.Chord A3 D4>
        {3.0} <music21.chord.Chord A3 D4>
        {4.0} <music21.chord.Chord A3 D4>
        {5.0} <music21.chord.Chord A3 C4 D4>
        {6.0} <music21.chord.Chord A3 C4 D4>
        {6.5} <music21.chord.Chord A3 C4 D4>
        {7.5} <music21.chord.Chord A3 C4 D4>
        {8.0} <music21.chord.Chord A3 C4 D4>
        {9.0} <music21.chord.Chord A3 C4 D4>
        {10.0} <music21.chord.Chord B3 D4>
        {11.0} <music21.chord.Chord B3 D4>
        {11.5} <music21.chord.Chord B3 D4>
        {12.5} <music2

In [13]:
for i in midi.parts: #jitne instrument mai vo part mai divide ho jata hai
    print(i)
    print(i[0])

<music21.stream.Part 0x7f4ab3424a20>
Piano


In [14]:
for i in midi.flat: #flat list of list ko list mai kardega so that hum usse access kar sake
    print(i)

Piano
<music21.tempo.MetronomeMark Quarter=163.0>
<music21.tempo.MetronomeMark Quarter=163.0>
d minor
<music21.meter.TimeSignature 5/4>
<music21.chord.Chord A3 D4>
<music21.note.Note D>
<music21.chord.Chord A3 D4>
<music21.chord.Chord A3 D4>
<music21.chord.Chord A3 D4>
<music21.chord.Chord A3 D4>
<music21.chord.Chord A3 D4>
<music21.chord.Chord A3 C4 D4>
<music21.note.Note D>
<music21.chord.Chord A3 C4 D4>
<music21.chord.Chord A3 C4 D4>
<music21.chord.Chord A3 C4 D4>
<music21.chord.Chord A3 C4 D4>
<music21.chord.Chord A3 C4 D4>
<music21.chord.Chord B3 D4>
<music21.note.Note D>
<music21.chord.Chord B3 D4>
<music21.chord.Chord B3 D4>
<music21.chord.Chord B3 D4>
<music21.chord.Chord B3 D4>
<music21.chord.Chord B3 D4>
<music21.chord.Chord B-3 D4>
<music21.note.Note D>
<music21.chord.Chord B-3 D4>
<music21.chord.Chord B-3 D4>
<music21.chord.Chord B-3 D4>
<music21.chord.Chord B-3 D4>
<music21.chord.Chord B-3 D4>
<music21.chord.Chord A3 D4>
<music21.note.Note D>
<music21.chord.Chord A3 D4>
<m

In [15]:
midi.flat.notes.show('text')

{0.0} <music21.chord.Chord A3 D4>
{0.0} <music21.note.Note D>
{1.0} <music21.chord.Chord A3 D4>
{1.5} <music21.chord.Chord A3 D4>
{2.5} <music21.chord.Chord A3 D4>
{3.0} <music21.chord.Chord A3 D4>
{4.0} <music21.chord.Chord A3 D4>
{5.0} <music21.chord.Chord A3 C4 D4>
{5.0} <music21.note.Note D>
{6.0} <music21.chord.Chord A3 C4 D4>
{6.5} <music21.chord.Chord A3 C4 D4>
{7.5} <music21.chord.Chord A3 C4 D4>
{8.0} <music21.chord.Chord A3 C4 D4>
{9.0} <music21.chord.Chord A3 C4 D4>
{10.0} <music21.chord.Chord B3 D4>
{10.0} <music21.note.Note D>
{11.0} <music21.chord.Chord B3 D4>
{11.5} <music21.chord.Chord B3 D4>
{12.5} <music21.chord.Chord B3 D4>
{13.0} <music21.chord.Chord B3 D4>
{14.0} <music21.chord.Chord B3 D4>
{15.0} <music21.chord.Chord B-3 D4>
{15.0} <music21.note.Note D>
{16.0} <music21.chord.Chord B-3 D4>
{16.5} <music21.chord.Chord B-3 D4>
{17.5} <music21.chord.Chord B-3 D4>
{18.0} <music21.chord.Chord B-3 D4>
{19.0} <music21.chord.Chord B-3 D4>
{20.0} <music21.chord.Chord A3 D4>

In [16]:
len(midi.flat.notes)

618

In [18]:
notes_to_parse = midi.flat.notes

In [20]:
for i in notes_to_parse:
    print(i, i.offset)

<music21.chord.Chord A3 D4> 0.0
<music21.note.Note D> 0.0
<music21.chord.Chord A3 D4> 1.0
<music21.chord.Chord A3 D4> 1.5
<music21.chord.Chord A3 D4> 2.5
<music21.chord.Chord A3 D4> 3.0
<music21.chord.Chord A3 D4> 4.0
<music21.chord.Chord A3 C4 D4> 5.0
<music21.note.Note D> 5.0
<music21.chord.Chord A3 C4 D4> 6.0
<music21.chord.Chord A3 C4 D4> 6.5
<music21.chord.Chord A3 C4 D4> 7.5
<music21.chord.Chord A3 C4 D4> 8.0
<music21.chord.Chord A3 C4 D4> 9.0
<music21.chord.Chord B3 D4> 10.0
<music21.note.Note D> 10.0
<music21.chord.Chord B3 D4> 11.0
<music21.chord.Chord B3 D4> 11.5
<music21.chord.Chord B3 D4> 12.5
<music21.chord.Chord B3 D4> 13.0
<music21.chord.Chord B3 D4> 14.0
<music21.chord.Chord B-3 D4> 15.0
<music21.note.Note D> 15.0
<music21.chord.Chord B-3 D4> 16.0
<music21.chord.Chord B-3 D4> 16.5
<music21.chord.Chord B-3 D4> 17.5
<music21.chord.Chord B-3 D4> 18.0
<music21.chord.Chord B-3 D4> 19.0
<music21.chord.Chord A3 D4> 20.0
<music21.note.Note D> 20.0
<music21.chord.Chord A3 D4> 21

In [21]:
notes_to_parse[1].pitch #notes ka attribute hota hai pitch(it defines frequency like D5 is higher frequency)

<music21.pitch.Pitch D3>

In [22]:
notes_to_parse[0]

<music21.chord.Chord A3 D4>

In [38]:
#list banayenge notes ki node hua toh list mai daal denge chord hua toh uski puri sequence add karni padegi
notes_demo = []  #list

#we have to handle 2 cases
for element in notes_to_parse:
    
    #if element is note
    if isinstance(element, note.Note): #obj iss class ka hai ya nahi --> isinstance
        notes_demo.append(str(element.pitch))
    
    #if element is chord
    elif isinstance(element, chord.Chord):
        notes_demo.append('+'.join(str(e) for e in element.normalOrder))
        

In [39]:
isinstance(notes_to_parse[1], note.Note) # to check output

True

In [40]:
#notes_to_parse[1].pitch
#str(notes_to_parse[1].pitch)
#notes_to_parse[1].name
#notes_to_parse[1].pitches
notes_to_parse[0].normalOrder #to check output

[9, 2]

In [41]:
notes_to_parse[0] #to check output

<music21.chord.Chord A3 D4>

In [42]:
"+".join( str(e) for e in notes_to_parse[0].normalOrder)

'9+2'

In [43]:
len(notes_demo) #to check output

618

In [45]:
len(set(notes_demo)) #unique value

55

In [47]:
notes = []

for file in glob.glob("midi_songs/*.mid"): 
    midi = converter.parse(file) #convert the file into stream.Score object
    
    print("Parsing %s" % file)
    
    notes_to_parse = None
    
    
    #unroll / flat the elements (notes/chords) into the list, because elements
    notes_to_parse = midi.flat.notes
    
    
    # At this point, we have notes_to_parse, which is a Iterator for all the Notes/Chords
    
    
    
    
    for element in notes_to_parse:
        
        # If the element is a Note, then store it's pitch
        if isinstance(element, note.Note): 
            notes.append(str(element.pitch))
            
        # If the element is a Chord, split each note of the chord and join them with +
        elif isinstance(element, chord.Chord): 
            notes.append('+'.join(str(n) for n in element.normalOrder))
    
    

Parsing midi_songs/Final_Fantasy_7_-_Judgement_Day_Piano.mid
Parsing midi_songs/sera_.mid
Parsing midi_songs/traitor.mid
Parsing midi_songs/ViviinAlexandria.mid
Parsing midi_songs/goldsaucer.mid
Parsing midi_songs/ff4-town.mid
Parsing midi_songs/FFVII_BATTLE.mid
Parsing midi_songs/Gold_Silver_Rival_Battle.mid
Parsing midi_songs/DOS.mid
Parsing midi_songs/Oppressed.mid
Parsing midi_songs/FFIXQuMarshP.mid
Parsing midi_songs/FFIII_Edgar_And_Sabin_Piano.mid
Parsing midi_songs/ff6shap.mid
Parsing midi_songs/balamb.mid
Parsing midi_songs/ff11_awakening_piano.mid
Parsing midi_songs/Finalfantasy6fanfarecomplete.mid
Parsing midi_songs/FF3_Third_Phase_Final_(Piano).mid
Parsing midi_songs/dontbeafraid.mid
Parsing midi_songs/cosmo.mid
Parsing midi_songs/tpirtsd-piano.mid
Parsing midi_songs/fortresscondor.mid
Parsing midi_songs/rufus.mid
Parsing midi_songs/0fithos.mid
Parsing midi_songs/costadsol.mid
Parsing midi_songs/Rachel_Piano_tempofix.mid
Parsing midi_songs/ff7-mainmidi.mid
Parsing midi_songs

In [51]:
with open('data/notes_easy', 'wb') as filepath:
    pickle.dump(notes, filepath)

In [52]:
with open('data/notes_easy', 'rb') as filepath:
    notes = pickle.load(filepath)

In [53]:
n_vocab = len(set(notes))

In [54]:
print("Total notes- ",len(notes))
print("Unique Notes- ",n_vocab)

Total notes-  60498
Unique Notes-  359


In [55]:
# notes is a list of all the notes in each music file - 60000
print(notes[:100])

['B2', 'B2', 'B2', 'B2', 'F#2', 'B2', 'B2', 'B2', 'B2', 'F#2', '9+2', 'B2', '8+1', 'B2', 'B2', 'B2', 'F#2', '9+2', 'B2', '11+4', 'B2', 'B2', 'B2', '9+2', 'F#2', 'B2', '6+11', 'B2', 'B2', 'B2', 'F#2', 'B2', 'B2', 'B2', 'B2', 'F#2', '9+2', 'B2', '8+1', 'B2', 'B2', 'B2', 'F#2', '9+2', 'B2', '11+4', 'B2', 'B2', 'B2', '1+5', 'F#2', 'B2', '2+6', 'B2', 'B2', 'B2', 'F#2', 'B2', 'B2', 'B2', 'B2', 'F#2', 'B2', 'B2', '6+11', 'B2', '1+6', '11+2', 'B2', '11+4', 'A2', '1+6', '1+6', 'A2', '4+9', 'A2', 'A2', 'E3', 'E3', '1+6', 'E3', '2+7', '4+9', 'E3', '6+11', 'B2', '1+4', 'B2', '2', 'B2', 'B2', 'F#2', 'B2', 'B2', 'B2', 'B2', 'F#2', 'B2', 'B2', 'B2']


# Prepare sequential data for LSTM

In [65]:
#  How many elements LSTM input should consider
sequence_length = 100

In [66]:
# ALl the unique elements in a sorted manner
pitchnames = sorted(set(item for item in notes))

In [67]:
# Mapping between note to int value
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
print(note_to_int)

{'0': 0, '0+1': 1, '0+1+5': 2, '0+1+6': 3, '0+2': 4, '0+2+3+7': 5, '0+2+4+7': 6, '0+2+5': 7, '0+2+6': 8, '0+2+7': 9, '0+3': 10, '0+3+5': 11, '0+3+5+8': 12, '0+3+6': 13, '0+3+6+8': 14, '0+3+6+9': 15, '0+3+7': 16, '0+4': 17, '0+4+5': 18, '0+4+6': 19, '0+4+7': 20, '0+5': 21, '0+5+6': 22, '0+6': 23, '1': 24, '1+2': 25, '1+2+4+6+8+10': 26, '1+2+6': 27, '1+2+6+8': 28, '1+3': 29, '1+3+5': 30, '1+3+5+8': 31, '1+3+6': 32, '1+3+7': 33, '1+3+8': 34, '1+4': 35, '1+4+6': 36, '1+4+6+9': 37, '1+4+7': 38, '1+4+7+10': 39, '1+4+7+9': 40, '1+4+8': 41, '1+5': 42, '1+5+8': 43, '1+5+9': 44, '1+6': 45, '1+7': 46, '10': 47, '10+0': 48, '10+0+2+5': 49, '10+0+3': 50, '10+0+4': 51, '10+0+5': 52, '10+1': 53, '10+1+3': 54, '10+1+3+5+6': 55, '10+1+3+6': 56, '10+1+4': 57, '10+1+4+6': 58, '10+1+5': 59, '10+11': 60, '10+11+3': 61, '10+11+3+5': 62, '10+2': 63, '10+2+3': 64, '10+2+4': 65, '10+2+5': 66, '10+3': 67, '11': 68, '11+0': 69, '11+0+4': 70, '11+0+4+6': 71, '11+0+4+7': 72, '11+0+5': 73, '11+1': 74, '11+1+4': 75,

In [68]:
network_input = []
network_output = []

In [69]:
# Make the data for LSTM Network
# Each Node will contain 100 input units
# Output will be the next unit in notes list

for i in range(len(notes) - sequence_length):
    
    sequence_in = notes[i:i + sequence_length] # contains 100 values 
    sequence_out = notes[i + sequence_length] # containes next values for these 100's
    
    # Since NN works with numeric data only, append the int values for inputs and outputs.
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

In [70]:
# No. of examples for our network

n_patterns = len(network_input)
print(n_patterns)

60398


In [71]:
# ight now network_input is lits, but reshape it into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
print(network_input.shape)

(60398, 100, 1)


In [72]:
normalised_network_input = network_input/float(n_vocab)

In [75]:
# network_output are the classes, encode one_hot_vector
network_output = np_utils.to_categorical(network_output)

In [76]:
print(network_output.shape)
print(normalised_network_input.shape)

(60398, 359)
(60398, 100, 1)


# Create Model

In [85]:
""" create the structure of the neural network """
model = Sequential()
model.add(LSTM(512,
            input_shape=(network_input.shape[1], network_input.shape[2]),
            return_sequences=True ))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [86]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 100, 512)          1052672   
_________________________________________________________________
dropout_10 (Dropout)         (None, 100, 512)          0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 100, 512)          2099200   
_________________________________________________________________
dropout_11 (Dropout)         (None, 100, 512)          0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 512)               2099200   
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_12 (Dropout)         (None, 256)               0         
__________

""" train the neural network """

filepath = "model/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"

checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min')

callbacks_list = [checkpoint]

model_his = model.fit(network_input, network_output, epochs=100, batch_size=64, callbacks=callbacks_list)

In [ ]:
model = load_model("model/weights-improvement-60-1.1406-bigger.hdf5")

# Predictions

In [ ]:
#  This is done beacuse we need network_input as list, and we had converted it into ndarray to feed into network
#  so, to get back the list.

sequence_length = 100
network_input = []
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])

In [ ]:
""" Generate notes from the neural network based on a sequence of notes """

# pick a random sequence from the input as a starting point for the prediction
start = np.random.randint(0, len(network_input)-1)

# Mapping from int to note
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

# initial pattern
pattern = network_input[start]
prediction_output = []

# generate 200 notes
for note_index in range(200):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)

    prediction = model.predict(prediction_input, verbose=0)

    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)

    
    # Remove the first value, and append the recent value.. 
    # This way input is moving forward step-by-step with time..
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [ ]:
#  Our newly generated song (mix of chords and notes)
print(prediction_output)

# Create Midi File

In [ ]:
""" convert the output from the prediction to notes and create a midi file from the notes """

offset = 0 # Time
output_notes = []

# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    
    # if pattern is a chord
    if ('+' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('+') # split all the notes from a chord
        notes = []
        for current_note in notes_in_chord: 
            new_note = note.Note(int(current_note)) # create Note object for each note in the chord
            new_note.storedInstrument = instrument.Piano() 
            notes.append(new_note) # list of Notes()
            
        new_chord = chord.Chord(notes)  # Create the Chord() from the list of notes.
        new_chord.offset = offset # set offset to the element
        output_notes.append(new_chord) 
        
    # if pattern is a note
    else:
        new_note = note.Note(pattern) #  create Note object
        new_note.offset = offset # set offset  (basically timestamp)
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note) 

    # increase offset each iteration so that notes do not stack (time stamp)
    offset += 0.5

In [ ]:
output_notes[:50]

In [ ]:
#  Create a Steam Object from our generated Notes, and write to the file. 
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='output_songs/test_output.mid')

## Let's play our music

In [ ]:
midi_stream.show('midi')